### 準備

In [1]:
import math

import matplotlib.figure as figure
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
import pandas as pd
import functions
from scipy.spatial.distance import cdist
from sklearn import metrics
from sklearn import svm
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.neighbors import NearestNeighbors

In [2]:
# データセットの読み込み
dataset = pd.read_csv('debutanizer_y_10.csv', encoding='SHIFT-JIS', index_col=0)
y = dataset.iloc[:, 0:1]
x_tmp = dataset.iloc[:, 1:]

dataset

,y,x1,x2,x3,x4,x5,x6,x7
sample_1,0.180,0.269,0.651,0.833,0.583,0.785,0.843,0.822
sample_2,0.180,0.268,0.650,0.852,0.578,0.776,0.839,0.822
sample_3,0.180,0.268,0.660,0.824,0.572,0.765,0.808,0.786
sample_4,0.180,0.267,0.668,0.808,0.566,0.753,0.800,0.786
sample_5,0.180,0.267,0.647,0.762,0.560,0.745,0.773,0.746
...,...,...,...,...,...,...,...,...
sample_2390,0.295,0.285,0.646,0.689,0.355,0.616,0.609,0.501
sample_2391,0.179,0.266,0.666,0.666,0.354,0.589,0.582,0.509
sample_2392,0.179,0.247,0.666,0.665,0.353,0.561,0.602,0.523
sample_2393,0.179,0.229,0.673,0.677,0.353,0.531,0.637,0.538


In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# サブプロット作成
fig = make_subplots(rows=len(dataset.columns), cols=1, shared_xaxes=True, 
                    subplot_titles=dataset.columns, vertical_spacing=0.05)

# 各列に対して折れ線グラフを追加
for i, column in enumerate(dataset.columns):
    fig.add_trace(
        go.Scatter(x=dataset.index, y=dataset[column], mode="lines+markers", name=column),
        row=i + 1, col=1
    )

# レイアウト設定
fig.update_layout(
    height=300 * len(dataset.columns),  # グラフ全体の高さを動的に設定
    title="Line Plots for Each Column",
    showlegend=False  # 全体の凡例を非表示に（個別の凡例がサブプロットに表示されるため）
)

# 描画
fig.show()

### モデル選択

In [ ]:
number_of_test_samples = 1000  # テストデータのサンプル数
method_name = 'lwpls'  # ソフトセンサーの種類は以下の通りです
# 'pls' : 最初のトレーニングデータで構築した PLS モデル (適応的ソフトセンサーではありません)
# 'svr' : 最初のトレーニングデータで構築した SVR モデル (適応的ソフトセンサーではありません)
# 'mwpls' : Moving Window PLSs
# 'mwsvr' : Moving Window SVR
# 'jitpls' : Just-In-Time PLS
# 'jitsvr' : Just-In-Time SVR
# 'lwpls' : Locally-Weighted PLS

number_of_samples_in_modeling = 100  # MW や JIT モデルにおけるモデル構築用サンプルの数 
y_measurement_delay = 10  # y の測定時間の遅れ（初期：10）
dynamics_max = 10  # いくつまで時間遅れ変数を追加するか。0 なら時間遅れ変数は追加されません
dynamics_span = 2  # いくつずつ時間を遅らせた変数を追加するか
add_nonlinear_terms_flag = False  # True (二乗項・交差項を追加) or False (追加しない)

max_sample_size = 10000  # データベースにおける最大のサンプル数
fold_number = 5  # N-fold CV の N
max_number_of_principal_components = 20  # 使用する主成分の最大数
svr_cs = 2 ** np.arange(-5, 11, dtype=float)  # C の候補
svr_epsilons = 2 ** np.arange(-10, 1, dtype=float)  # ε の候補
svr_gammas = 2 ** np.arange(-20, 11, dtype=float)  # γ の候補
lwpls_lambdas = 2 ** np.arange(-9, 6, dtype=float)

| モデル        | 特徴                                     | 用途・効果                                    |
|---------------|------------------------------------------|---------------------------------------------|
| `mwpls`       | 窓内データでPLSを適用                   | データの一部で局所的な関係性を捉える         |
| `mwsvr`       | 窓内データでSVRを適用                   | 非線形な局所関係やノイズ耐性が必要な場合     |
| `jitpls`      | 予測ごとに近いデータを選びPLSを適用     | リアルタイムに最適な分析を行いたい場合      |
| `jitsvr`      | 予測ごとに近いデータを選びSVRを適用     | 非線形性やノイズを考慮した柔軟な予測        |
| `lwpls`       | 全データに重みを付け、PLSを適用         | 近くのデータに重点を置いた予測が必要な場合  |

### PLS（Partial Least Squares, 部分最小二乗法）
- **特徴**:  
  データの説明変数（X）と目的変数（Y）の間の関係を探す回帰手法。多次元データでも使いやすく、相関の強いデータやサンプル数が少ない場合にも適している。  
- **用途**:  
  化学やバイオ分野でのスペクトル解析や、複数の要因が絡む現象のモデリングなどに使われる。  
- **ポイント**:  
  データを「主成分」に圧縮しながら回帰分析を行うので、ノイズに強く、過学習を抑えやすい。

---

### SVR（Support Vector Regression, サポートベクター回帰）
- **特徴**:  
  サポートベクターマシン（SVM）を使った回帰手法。非線形の関係もカーネル法によってモデル化できるのが特徴。  
- **用途**:  
  時系列予測、需要予測、株価分析など、複雑な非線形関係を含むデータに適している。  
- **ポイント**:  
  データの一部（サポートベクター）のみを使ってモデルを作るため、計算効率が良い。また、パラメータ（C, ε, γ）の選択がモデル性能に大きく影響する。

### JIT (Just-In-Time) モデルのイメージと具体例

**イメージ**:  
JIT モデルは、「必要な時に、その場で作るカスタマイズ型のモデル」と考えると分かりやすいです。通常の機械学習モデルは一度学習してから、同じモデルをすべての新しいデータに適用します。一方、JIT モデルは、新しいデータが来るたびに、そのデータに最適なモデルを作成します。

---

### **具体例: 工場の異常検知**
**背景**:  
ある工場で、複数の機械が稼働しています。それぞれの機械の状態（温度、圧力、振動など）をモニタリングして、異常を検知したいとします。  
- 例えば、「機械A」と「機械B」では動作環境が異なるため、同じモデルを使うと精度が低下する可能性があります。  
- さらに、同じ「機械A」でも、季節や稼働年数によって特徴が変わることがあります。

**通常のモデル**:  
- 全体のデータを用いて1つの大きな異常検知モデルを作成し、すべての機械に適用します。
  - **問題点**: 機械や環境の違いを考慮しにくい。

**JITモデル**:  
- 新しいデータ（例えば「機械A」の現在の状態）が入ってきたら、その時点のデータを基に「機械A」に特化した小さなモデルを作ります。  
- このモデルは、過去の「機械A」に近いデータを中心に学習します。

---

### **JITモデリングの流れ**
1. **新しいデータが来る**:  
   例: 「現在の機械Aの状態データ」が測定される。  
   
2. **近いデータを選ぶ**:  
   「過去のデータ」の中から、現在のデータに似ているものを選び出します（距離計算などで）。  

3. **モデルを作る**:  
   選んだデータを用いて、その場でローカルなモデルを構築します（PLSやSVRなど）。  

4. **予測・判断**:  
   作成したモデルを用いて、新しいデータを予測・評価します。

---

### **例: 振動の異常検知**
- **新しいデータ**: 機械Aの振動が「12.3Hz」だった。  
- **過去のデータ**: 機械Aの振動履歴が保存されている。
  - 例えば、10Hz～15Hzの間で動作していた記録が中心。  
- **データ選択**: 現在の「12.3Hz」に近いデータだけを選ぶ（例えば11～13Hz）。  
- **モデル構築**: 選んだデータで小さなモデルを作り、異常値を検出する。

---

### **メリット**
- 適応性が高い（データの変化に即座に対応）。  
- 無駄なデータを使わず、効率的なモデリングが可能。  

### **デメリット**
- 毎回モデルを作るので計算コストが高い。  
- データ選択やパラメータ調整が複雑になることがある。  

このアプローチは、**環境の変化が大きい場合**や、**個別に対応する必要がある問題**に特に有効です。

### 特徴量エンジニアリング

In [45]:
# 説明変数の非線形変換
if add_nonlinear_terms_flag:
    x_tmp = functions.add_nonlinear_terms(x_tmp)  # 説明変数の二乗項や交差項を追加
    x = x_tmp.drop(x_tmp.columns[x_tmp.std() == 0], axis=1)  # 標準偏差が 0 の説明変数を削除
else:
    x = x_tmp.copy()
x

,x1,x2,x3,x4,x5,x6,x7
sample_1,0.269,0.651,0.833,0.583,0.785,0.843,0.822
sample_2,0.268,0.650,0.852,0.578,0.776,0.839,0.822
sample_3,0.268,0.660,0.824,0.572,0.765,0.808,0.786
sample_4,0.267,0.668,0.808,0.566,0.753,0.800,0.786
sample_5,0.267,0.647,0.762,0.560,0.745,0.773,0.746
...,...,...,...,...,...,...,...
sample_2390,0.285,0.646,0.689,0.355,0.616,0.609,0.501
sample_2391,0.266,0.666,0.666,0.354,0.589,0.582,0.509
sample_2392,0.247,0.666,0.665,0.353,0.561,0.602,0.523
sample_2393,0.229,0.673,0.677,0.353,0.531,0.637,0.538


- flag = False だと、そのままコピー

In [46]:
# 時間遅れ変数の追加
dataset_with_dynamics = functions.add_time_delayed_variable(pd.concat([y, x], axis=1), dynamics_max, dynamics_span)

dataset_with_dynamics

array([[0.155, 0.242, 0.263, ..., 0.746, 0.786, 0.822],
       [0.155, 0.231, 0.253, ..., 0.786, 0.786, 0.822],
       [0.155, 0.22 , 0.242, ..., 0.821, 0.746, 0.786],
       ...,
       [0.179, 0.247, 0.285, ..., 0.587, 0.627, 0.665],
       [0.179, 0.229, 0.266, ..., 0.555, 0.574, 0.642],
       [0.179, 0.216, 0.247, ..., 0.554, 0.587, 0.627]])

In [47]:
# トレーニングデータとテストデータに分割
dataset_train_with_dynamics = dataset_with_dynamics[:-number_of_test_samples, :]
dataset_test_with_dynamics = dataset_with_dynamics[-number_of_test_samples:, :]

# トレーニングデータでは y が測定されたサンプルのみ収集
y_measured_dataset_train = dataset_train_with_dynamics[0:1, :]
for sample_number in range(1, dataset_train_with_dynamics.shape[0]):
    if y_measured_dataset_train[-1, 0] != dataset_train_with_dynamics[sample_number, 0]:
        y_measured_dataset_train = np.r_[
            y_measured_dataset_train, dataset_train_with_dynamics[sample_number:sample_number + 1, :]]
y_measured_dataset_train = np.delete(y_measured_dataset_train, 0, 0)
y_measured_dataset_train = pd.DataFrame(y_measured_dataset_train)
y_train = y_measured_dataset_train.iloc[:, 0]
x_train = y_measured_dataset_train.iloc[:, 1:]

y_measured_dataset_train

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0.1960,0.2340,0.162,0.2080,0.201,0.220,0.2420,0.669,0.648,0.647,...,0.812,0.749,0.728,0.801,0.766,0.772,0.798,0.718,0.720,0.789
1,0.2490,0.0728,0.243,0.2410,0.239,0.236,0.2340,0.643,0.682,0.658,...,0.758,0.780,0.770,0.766,0.793,0.775,0.763,0.769,0.759,0.766
2,0.2810,0.1740,0.139,0.0554,0.131,0.140,0.0728,0.654,0.670,0.638,...,0.744,0.813,0.826,0.780,0.763,0.774,0.732,0.813,0.808,0.793
3,0.3090,0.1040,0.102,0.1010,0.203,0.254,0.1740,0.682,0.672,0.688,...,0.766,0.855,0.809,0.771,0.737,0.739,0.759,0.863,0.814,0.763
4,0.3760,0.1510,0.139,0.1280,0.116,0.107,0.1040,0.663,0.638,0.684,...,0.780,0.799,0.737,0.735,0.780,0.781,0.740,0.821,0.762,0.737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0.0202,0.2310,0.235,0.3650,0.526,0.631,0.7360,0.638,0.667,0.647,...,0.221,0.485,0.520,0.574,0.206,0.134,0.206,0.450,0.462,0.508
134,0.2530,0.2770,0.262,0.2460,0.231,0.227,0.2310,0.665,0.657,0.695,...,0.410,0.438,0.335,0.239,0.403,0.409,0.372,0.420,0.292,0.206
135,0.4910,0.3720,0.348,0.3250,0.308,0.293,0.2770,0.697,0.689,0.667,...,0.449,0.435,0.422,0.431,0.492,0.414,0.414,0.409,0.373,0.403
136,0.4190,0.3390,0.356,0.3730,0.390,0.396,0.3720,0.661,0.676,0.689,...,0.417,0.437,0.534,0.530,0.436,0.434,0.362,0.402,0.479,0.492


In [48]:
# テストデータ
dataset_test_with_dynamics = pd.DataFrame(dataset_test_with_dynamics)
y_test_all = dataset_test_with_dynamics.iloc[:, 0]
x_test = dataset_test_with_dynamics.iloc[:, 1:]

In [49]:
# サンプルの調整
if method_name[0:2] == 'mw':
    y_train = y_train.iloc[-number_of_samples_in_modeling:]
    x_train = x_train.iloc[-number_of_samples_in_modeling:, :]
else:
    y_train = y_train.iloc[-max_sample_size:]
    x_train = x_train.iloc[-max_sample_size:, :]
    if method_name[0:3] == 'jit':
        nn_model = NearestNeighbors(metric='euclidean')  # サンプル選択用の k-NN モデルの宣言

コード全体の目的

サンプル選択の方法を動的に変えることで、適切なデータをモデルに供給します。

使用手法の概要：

mw：直近の一定数のデータを使用する移動窓法。

その他：指定された最大サンプルサイズのデータを使用。

jit：ユークリッド距離を用いて、最も関連性の高いデータを選択する Just-In-Time モデリング。

In [50]:
# オートスケーリング    
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std()
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std()
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std()

### 学習

In [51]:
# ハイパーパラメータの最適化やモデリング
if method_name == 'pls' or method_name == 'mwpls':
    # CV による成分数の最適化
    components = []  # 空の list の変数を作成して、成分数をこの変数に追加していきます同じく成分数をこの変数に追加
    r2_in_cv_all = []  # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加
    for component in range(1, min(np.linalg.matrix_rank(autoscaled_x_train), max_number_of_principal_components) + 1):
        # PLS
        model = PLSRegression(n_components=component)  # PLS モデルの宣言
        estimated_y_in_cv = pd.DataFrame(cross_val_predict(model, autoscaled_x_train, autoscaled_y_train,
                                                           cv=fold_number))  # クロスバリデーション推定値の計算し、DataFrame型に変換
        estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean()  # スケールをもとに戻す
        r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv)  # r2 を計算
        print(component, r2_in_cv)  # 成分数と r2 を表示
        r2_in_cv_all.append(r2_in_cv)  # r2 を追加
        components.append(component)  # 成分数を追加
    optimal_component_number = components[r2_in_cv_all.index(max(r2_in_cv_all))]  # 最適成分数
    # PLS
    model = PLSRegression(n_components=optimal_component_number)  # モデルの宣言
    model.fit(autoscaled_x_train, autoscaled_y_train)  # モデルの構築
elif method_name == 'svr' or method_name == 'mwsvr' or method_name == 'jitsvr':
    # グラム行列の分散を最大化することによる γ の最適化
    variance_of_gram_matrix = list()
    for svr_gamma in svr_gammas:
        gram_matrix = np.exp(
            -svr_gamma * cdist(autoscaled_x_train, autoscaled_x_train, metric='seuclidean'))
        variance_of_gram_matrix.append(gram_matrix.var(ddof=1))
    optimal_svr_gamma = svr_gammas[np.where(variance_of_gram_matrix == np.max(variance_of_gram_matrix))[0][0]]
    # CV による ε の最適化
    model_in_cv = GridSearchCV(svm.SVR(kernel='rbf', C=3, gamma=optimal_svr_gamma), {'epsilon': svr_epsilons},
                               cv=fold_number)
    model_in_cv.fit(autoscaled_x_train, autoscaled_y_train)
    optimal_svr_epsilon = model_in_cv.best_params_['epsilon']
    # CV による C の最適化
    model_in_cv = GridSearchCV(svm.SVR(kernel='rbf', epsilon=optimal_svr_epsilon, gamma=optimal_svr_gamma),
                               {'C': svr_cs}, cv=fold_number)
    model_in_cv.fit(autoscaled_x_train, autoscaled_y_train)
    optimal_svr_c = model_in_cv.best_params_['C']
    # CV による γ の最適化
    model_in_cv = GridSearchCV(svm.SVR(kernel='rbf', epsilon=optimal_svr_epsilon, C=optimal_svr_c),
                               {'gamma': svr_gammas}, cv=fold_number)
    model_in_cv.fit(autoscaled_x_train, autoscaled_y_train)
    optimal_svr_gamma = model_in_cv.best_params_['gamma']
    # 最適化された C, ε, γ
    print('C : {0}\nε : {1}\nGamma : {2}'.format(optimal_svr_c, optimal_svr_epsilon, optimal_svr_gamma))
    # SVR
    model = svm.SVR(kernel='rbf', C=optimal_svr_c, epsilon=optimal_svr_epsilon, gamma=optimal_svr_gamma)  # モデルの宣言
    model.fit(autoscaled_x_train, autoscaled_y_train)  # モデルの構築
elif method_name == 'lwpls':
    # CV によるハイパーパラメータの最適化
    autoscaled_x_train = np.array(autoscaled_x_train)
    autoscaled_y_train = np.array(autoscaled_y_train)
    y_train_array = np.array(y_train)
    r2cvs = np.empty(
        (min(np.linalg.matrix_rank(autoscaled_x_train), max_number_of_principal_components), len(lwpls_lambdas)))
    min_number = math.floor(x_train.shape[0] / fold_number)
    mod_numbers = x_train.shape[0] - min_number * fold_number
    index = np.matlib.repmat(np.arange(1, fold_number + 1, 1), 1, min_number).ravel()
    if mod_numbers != 0:
        index = np.r_[index, np.arange(1, mod_numbers + 1, 1)]
    indexes_for_division_in_cv = np.random.permutation(index)
    np.random.seed()
    for parameter_number, lambda_in_similarity in enumerate(lwpls_lambdas):
        estimated_y_in_cv = np.empty((len(y_train_array), r2cvs.shape[0]))
        for fold in np.arange(1, fold_number + 1, 1):
            autoscaled_x_train_in_cv = autoscaled_x_train[indexes_for_division_in_cv != fold, :]
            autoscaled_y_train_in_cv = autoscaled_y_train[indexes_for_division_in_cv != fold]
            autoscaled_x_validation_in_cv = autoscaled_x_train[indexes_for_division_in_cv == fold, :]

            estimated_y_validation_in_cv = functions.lwpls(autoscaled_x_train_in_cv, autoscaled_y_train_in_cv,
                                                                  autoscaled_x_validation_in_cv, r2cvs.shape[0],
                                                                  lambda_in_similarity)
            estimated_y_in_cv[indexes_for_division_in_cv == fold, :] = estimated_y_validation_in_cv * y_train_array.std(
                ddof=1) + y_train_array.mean()

        estimated_y_in_cv[np.isnan(estimated_y_in_cv)] = 99999
        ss = (y_train_array - y_train_array.mean()).T.dot(y_train_array - y_train_array.mean())
        press = np.diag(
            (np.matlib.repmat(y_train_array.reshape(len(y_train_array), 1), 1,
                              estimated_y_in_cv.shape[1]) - estimated_y_in_cv).T.dot(
                np.matlib.repmat(y_train_array.reshape(len(y_train_array), 1), 1,
                                 estimated_y_in_cv.shape[1]) - estimated_y_in_cv))
        r2cvs[:, parameter_number] = 1 - press / ss

    best_candidate_number = np.where(r2cvs == r2cvs.max())

    optimal_component_number = best_candidate_number[0][0] + 1
    optimal_lambda_in_similarity = lwpls_lambdas[best_candidate_number[1][0]]

/Users/yanai/Desktop/lesson_py/Github/adaptive_soft_sensors/functions.py:109: RuntimeWarning:

invalid value encountered in true_divide

/Users/yanai/Desktop/lesson_py/Github/adaptive_soft_sensors/functions.py:110: RuntimeWarning:

invalid value encountered in true_divide

/Users/yanai/Desktop/lesson_py/Github/adaptive_soft_sensors/functions.py:116: RuntimeWarning:

invalid value encountered in true_divide

/Users/yanai/Desktop/lesson_py/Github/adaptive_soft_sensors/functions.py:116: RuntimeWarning:

divide by zero encountered in true_divide



#### pls
- 各成分数によるR2の数値が出力
- 一番数値が高い成分数を選択する必要あり

#### svr
- C, ε, Gamma のパラメータが出力

### 推論

In [52]:
# y の推定やモデリング
if method_name == 'pls' or method_name == 'svr':
    estimated_y_test_all = model.predict(autoscaled_x_test) * y_train.std() + y_train.mean()
else:
    estimated_y_test_all = np.zeros((len(y_test_all)))
    for test_sample_number in range(len(y_test_all)):
        print(test_sample_number + 1, '/', len(y_test_all))
        autoscaled_x_test = (x_test.iloc[
                             test_sample_number:test_sample_number + 1, ] - x_train.mean()) / x_train.std()  # オートスケーリング
        # y の推定
        if method_name[0:2] == 'mw':
            autoscaled_estimated_y_test_tmp = model.predict(autoscaled_x_test)
        elif method_name == 'lwpls':
            autoscaled_estimated_y_test_tmp = functions.lwpls(autoscaled_x_train, autoscaled_y_train,
                                                                     autoscaled_x_test, optimal_component_number,
                                                                     optimal_lambda_in_similarity)
            autoscaled_estimated_y_test_tmp = autoscaled_estimated_y_test_tmp[:, optimal_component_number - 1]
        elif method_name[0:3] == 'jit':
            # サンプル選択
            nn_model.fit(autoscaled_x_train)
            tmp, nn_index_test = nn_model.kneighbors(autoscaled_x_test,
                                                     n_neighbors=min(number_of_samples_in_modeling, x_train.shape[0]))
            x_train_jit = x_train.iloc[nn_index_test[0, :], :]
            y_train_jit = y_train.iloc[nn_index_test[0, :]]
            # オートスケーリング    
            autoscaled_x_train_jit = (x_train_jit - x_train_jit.mean()) / x_train_jit.std()
            autoscaled_y_train_jit = (y_train_jit - y_train_jit.mean()) / y_train_jit.std()
            autoscaled_x_test_jit = (x_test.iloc[
                                     test_sample_number:test_sample_number + 1, ] - x_train_jit.mean()) / x_train_jit.std()
            # ハイパーパラメータの最適化
            if method_name == 'jitpls':
                # CV による成分数の最適化
                components = []  # 空の list の変数を作成して、成分数をこの変数に追加していきます同じく成分数をこの変数に追加
                r2_in_cv_all = []  # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加
                for component in range(1, min(np.linalg.matrix_rank(autoscaled_x_train_jit),
                                              max_number_of_principal_components) + 1):
                    model = PLSRegression(n_components=component)  # PLS モデルの宣言
                    estimated_y_in_cv = pd.DataFrame(
                        cross_val_predict(model, autoscaled_x_train_jit, autoscaled_y_train_jit,
                                          cv=fold_number))  # クロスバリデーション推定値の計算し、DataFrame型に変換
                    estimated_y_in_cv = estimated_y_in_cv * y_train_jit.std() + y_train_jit.mean()  # スケールをもとに戻す
                    r2_in_cv = metrics.r2_score(y_train_jit, estimated_y_in_cv)  # r2 を計算
                    r2_in_cv_all.append(r2_in_cv)  # r2 を追加
                    components.append(component)  # 成分数を追加
                optimal_component_number = components[r2_in_cv_all.index(max(r2_in_cv_all))]  # 最適成分数
                # PLS
                model = PLSRegression(n_components=optimal_component_number)  # モデルの宣言
            # モデリング
            model.fit(autoscaled_x_train_jit, autoscaled_y_train_jit)  # モデルの構築
            autoscaled_estimated_y_test_tmp = model.predict(autoscaled_x_test_jit)  # 推定
        if np.isnan(autoscaled_estimated_y_test_tmp):
            if test_sample_number == 0:
                estimated_y_test_all[test_sample_number] = 0
            else:
                estimated_y_test_all[test_sample_number] = estimated_y_test_all[test_sample_number - 1]
        else:
            estimated_y_test_all[test_sample_number] = autoscaled_estimated_y_test_tmp * y_train.std() + y_train.mean()

        if test_sample_number - y_measurement_delay >= 1:
            if y_test_all[test_sample_number - y_measurement_delay] - y_test_all[
                test_sample_number - y_measurement_delay - 1] != 0:
                x_train = pd.concat([x_train, x_test.iloc[
                                              test_sample_number - y_measurement_delay:test_sample_number - y_measurement_delay + 1, ]],
                                    axis=0)
                y_train = pd.concat([y_train, y_test_all.iloc[test_sample_number - y_measurement_delay:test_sample_number - y_measurement_delay + 1]])
                # サンプルの調整
                if method_name[0:2] == 'mw':
                    y_train = y_train.iloc[-number_of_samples_in_modeling:]
                    x_train = x_train.iloc[-number_of_samples_in_modeling:, :]
                else:
                    y_train = y_train.iloc[-max_sample_size:]
                    x_train = x_train.iloc[-max_sample_size:, :]
                # オートスケーリング    
                autoscaled_x_train = (x_train - x_train.mean()) / x_train.std()
                autoscaled_y_train = (y_train - y_train.mean()) / y_train.std()
                # ハイパーパラメータの最適化
                if method_name == 'mwpls':
                    # CV による成分数の最適化
                    components = []  # 空の list の変数を作成して、成分数をこの変数に追加していきます同じく成分数をこの変数に追加
                    r2_in_cv_all = []  # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加
                    for component in range(1, min(np.linalg.matrix_rank(autoscaled_x_train),
                                                  max_number_of_principal_components) + 1):
                        model = PLSRegression(n_components=component)  # PLS モデルの宣言
                        estimated_y_in_cv = pd.DataFrame(
                            cross_val_predict(model, autoscaled_x_train, autoscaled_y_train,
                                              cv=fold_number))  # クロスバリデーション推定値の計算し、DataFrame型に変換
                        estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean()  # スケールをもとに戻す
                        r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv)  # r2 を計算
                        r2_in_cv_all.append(r2_in_cv)  # r2 を追加
                        components.append(component)  # 成分数を追加
                    optimal_component_number = components[r2_in_cv_all.index(max(r2_in_cv_all))]  # 最適成分数
                    # PLS
                    model = PLSRegression(n_components=optimal_component_number)  # モデルの宣言
                # モデリング
                if method_name[0:2] == 'mw':
                    model.fit(autoscaled_x_train, autoscaled_y_train)  # モデルの構築

1 / 1000
2 / 1000
3 / 1000
4 / 1000
5 / 1000
6 / 1000
7 / 1000
8 / 1000
9 / 1000
10 / 1000
11 / 1000
12 / 1000
13 / 1000
14 / 1000
15 / 1000
16 / 1000
17 / 1000
18 / 1000
19 / 1000
20 / 1000
21 / 1000
22 / 1000
23 / 1000
24 / 1000
25 / 1000
26 / 1000
27 / 1000
28 / 1000
29 / 1000
30 / 1000
31 / 1000
32 / 1000
33 / 1000
34 / 1000
35 / 1000
36 / 1000
37 / 1000
38 / 1000
39 / 1000
40 / 1000
41 / 1000
42 / 1000
43 / 1000
44 / 1000
45 / 1000
46 / 1000
47 / 1000
48 / 1000
49 / 1000
50 / 1000
51 / 1000
52 / 1000
53 / 1000
54 / 1000
55 / 1000
56 / 1000
57 / 1000
58 / 1000
59 / 1000
60 / 1000
61 / 1000
62 / 1000
63 / 1000
64 / 1000
65 / 1000
66 / 1000
67 / 1000
68 / 1000
69 / 1000
70 / 1000
71 / 1000
72 / 1000
73 / 1000
74 / 1000
75 / 1000
76 / 1000
77 / 1000
78 / 1000
79 / 1000
80 / 1000
81 / 1000
82 / 1000
83 / 1000
84 / 1000
85 / 1000
86 / 1000
87 / 1000
88 / 1000
89 / 1000
90 / 1000
91 / 1000
92 / 1000
93 / 1000
94 / 1000
95 / 1000
96 / 1000
97 / 1000
98 / 1000
99 / 1000
100 / 1000
101 / 10

/Users/yanai/Desktop/lesson_py/Github/adaptive_soft_sensors/functions.py:116: RuntimeWarning:

divide by zero encountered in true_divide

/Users/yanai/Desktop/lesson_py/Github/adaptive_soft_sensors/functions.py:116: RuntimeWarning:

divide by zero encountered in true_divide



228 / 1000
229 / 1000
230 / 1000
231 / 1000
232 / 1000
233 / 1000
234 / 1000
235 / 1000
236 / 1000
237 / 1000
238 / 1000
239 / 1000
240 / 1000
241 / 1000
242 / 1000
243 / 1000
244 / 1000
245 / 1000
246 / 1000
247 / 1000
248 / 1000
249 / 1000
250 / 1000
251 / 1000
252 / 1000
253 / 1000
254 / 1000
255 / 1000
256 / 1000
257 / 1000
258 / 1000
259 / 1000
260 / 1000
261 / 1000
262 / 1000
263 / 1000
264 / 1000
265 / 1000
266 / 1000
267 / 1000
268 / 1000
269 / 1000
270 / 1000
271 / 1000
272 / 1000
273 / 1000
274 / 1000
275 / 1000
276 / 1000
277 / 1000
278 / 1000
279 / 1000
280 / 1000
281 / 1000
282 / 1000
283 / 1000
284 / 1000
285 / 1000
286 / 1000
287 / 1000
288 / 1000
289 / 1000
290 / 1000
291 / 1000
292 / 1000
293 / 1000
294 / 1000
295 / 1000
296 / 1000
297 / 1000
298 / 1000
299 / 1000
300 / 1000
301 / 1000
302 / 1000
303 / 1000
304 / 1000
305 / 1000
306 / 1000
307 / 1000
308 / 1000
309 / 1000
310 / 1000
311 / 1000
312 / 1000
313 / 1000
314 / 1000
315 / 1000
316 / 1000
317 / 1000
318 / 1000

In [53]:
estimated_y_test_all = pd.DataFrame(estimated_y_test_all, index=x_test.index, columns=['estimated_y'])
estimated_y_test_all.to_csv('estimated_y_test.csv')  # 推定値を csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [54]:
# テストデータの y の実測値と推定値において y が測定されたサンプルのみ収集
ys_test = pd.concat([y_test_all, estimated_y_test_all], axis=1)
y_measured_ys_test = ys_test.iloc[0:1, :]
measured_index_test = []
for sample_number in range(1, ys_test.shape[0]):
    if y_measured_ys_test.iloc[-1, 0] != ys_test.iloc[sample_number, 0]:
        y_measured_ys_test = pd.concat([y_measured_ys_test, ys_test.iloc[sample_number:sample_number + 1, :]], axis=0)
        measured_index_test.append(sample_number)
y_measured_ys_test = y_measured_ys_test.drop(0, axis=0)

### 精度確認

In [55]:
import plotly.graph_objects as go

# 実測値と推定値
actual_y = y_measured_ys_test.iloc[:, 0]
estimated_y = y_measured_ys_test.iloc[:, 1]

# 実測値と推定値の最大値・最小値を取得
y_max = max(actual_y.max(), estimated_y.max())
y_min = min(actual_y.min(), estimated_y.min())

# プロット範囲を調整（-5%～+5%）
margin = 0.05 * (y_max - y_min)
plot_min = y_min - margin
plot_max = y_max + margin

# Figureの作成
fig = go.Figure()

# 実測値 vs. 推定値の散布図
fig.add_trace(go.Scatter(
    x=actual_y,
    y=estimated_y,
    mode='markers',
    marker=dict(color='blue'),
    name='実測値 vs. 推定値'
))

# 対角線の追加
fig.add_trace(go.Scatter(
    x=[plot_min, plot_max],
    y=[plot_min, plot_max],
    mode='lines',
    line=dict(color='black', dash='solid'),
    name='対角線'
))

# レイアウトの設定
fig.update_layout(
    title='実測値 vs. 推定値',
    xaxis=dict(title='actual y', range=[plot_min, plot_max]),
    yaxis=dict(title='estimated y', range=[plot_min, plot_max]),
    width=600,  # 正方形に近い形状
    height=600,
    font=dict(size=18)
)

# 描画
fig.show()

In [56]:
# テストデータのr2, RMSE, MAE
print('r^2 for test data :', metrics.r2_score(y_measured_ys_test.iloc[:, 0], y_measured_ys_test.iloc[:, 1]))
print('RMSE for test data :',
      metrics.mean_squared_error(y_measured_ys_test.iloc[:, 0], y_measured_ys_test.iloc[:, 1]) ** 0.5)
print('MAE for test data :', metrics.mean_absolute_error(y_measured_ys_test.iloc[:, 0], y_measured_ys_test.iloc[:, 1]))

r^2 for test data : 0.21295656997660817
RMSE for test data : 0.15782655648037486
MAE for test data : 0.10899387222359591


- 'mwpls'
- r^2 for test data : 0.4327039984724812
- RMSE for test data : 0.13399407448530026
- MAE for test data : 0.09587478712379717

- lwpls
- r^2 for test data : 0.19821343274765435
- RMSE for test data : 0.159297925478878
- MAE for test data : 0.11072985113831021

- mvsvr
- r^2 for test data : 0.3872078551453947
- RMSE for test data : 0.13926350676677476
- MAE for test data : 0.09473349970003155

- jitsvr
- r^2 for test data : 0.22551676014349187
- RMSE for test data : 0.15656213822227016
- MAE for test data : 0.11165214429819822

- lwpls
- r^2 for test data : 0.21295656997660817
- RMSE for test data : 0.15782655648037486
- MAE for test data : 0.10899387222359591

初期のデータセット、パラメーターでは一番 "mwpls" の精度が一番良い

In [57]:
import plotly.graph_objects as go

# 実測値と推定値
time = range(estimated_y_test_all.shape[0])  # 時間軸
estimated_y = estimated_y_test_all.iloc[:, 0]
actual_y_time = [time[i] for i in measured_index_test]
actual_y = y_measured_ys_test.iloc[:, 0]

# 図全体のプロット
fig_full = go.Figure()

# 推定値の時間プロット
fig_full.add_trace(go.Scatter(
    x=list(time),
    y=estimated_y,
    mode='lines+markers',
    name='estimated y',
    line=dict(color='blue'),
    marker=dict(color='blue')
))

# 実測値の時間プロット
fig_full.add_trace(go.Scatter(
    x=actual_y_time,
    y=actual_y,
    mode='markers',
    name='actual y',
    marker=dict(color='red', size=12)
))

# レイアウトの設定
fig_full.update_layout(
    title='Time Plot (Full Range)',
    xaxis=dict(title='time', range=[0, estimated_y_test_all.shape[0] + 1]),
    yaxis=dict(title=dataset.columns[0]),
    legend=dict(x=1.05, y=1, borderwidth=1, font=dict(size=18)),
    font=dict(size=18)
)

# 描画
fig_full.show()

# 拡大版のプロット
fig_zoomed = go.Figure(fig_full)  # 全体プロットをコピー
fig_zoomed.update_layout(
    title='Time Plot (Zoomed)',
    xaxis=dict(range=[600, 800])
)

# 描画
fig_zoomed.show()